In [1]:
import requests
from bs4 import BeautifulSoup
import pymongo
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [ ]:
class Database(object):
    def __init__(self):
        self.connection = pymongo.MongoClient("localhost", 27017)
        self.db = self.connection["DSS4_SmallP"]
        self.collection = self.db["Movie_mojo"]
        
def Load_Data_mojo_yearly_dom(page,year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?page={}&view=releasedate&view2=domestic&yr={}&p=.htm'.format(page,year)
    return requests.get(Mojo_url)
def Load_Data_mojo_yearly_intl(year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?view2=worldwide&yr={}&p=.htm'.format(year)
    return requests.get(Mojo_url)

def Load_Data_mojo_movie_intl(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=intl'.format(M_id)
    return requests.get(Mojo_url)


In [2]:
import json
f = open("movie.json", "r")
movie_data = json.load(f)
f.close()

In [3]:
print movie_data

[{u'realease_kor': u'8/5/10', u'Webpage': u'/movies/?id=toystory3.htm', u'Mname': u'Toy Story 3', u'Domestic': u'$415.0', u'budget': u'$200 million', u'gross_kor': 12248035.0, u'release_usa': u'June 18, 2010', u'year': 2010, u'runtime': 103.0, u'Oversea': u'$652.0', u'rating_usa': u'G'}, {u'realease_kor': u'3/4/10', u'Webpage': u'/movies/?id=aliceinwonderland10.htm', u'Mname': u'Alice in Wonderland (2010)', u'Domestic': u'$334.2', u'budget': u'$200 million', u'gross_kor': 17954373.0, u'release_usa': u'March 5, 2010', u'year': 2010, u'runtime': 109.0, u'Oversea': u'$691.3', u'rating_usa': u'PG'}, {u'realease_kor': u'12/16/10', u'Webpage': u'/movies/?id=harrypotter7.htm', u'Mname': u'Harry Potter and the Deathly Hallows Part 1', u'Domestic': u'$296.0', u'budget': u'N/A', u'gross_kor': 18084930.0, u'release_usa': u'November 19, 2010', u'year': 2010, u'runtime': 150.0, u'Oversea': u'$664.3', u'rating_usa': u'PG-13'}, {u'realease_kor': u'7/21/10', u'Webpage': u'/movies/?id=inception.htm', u